In [14]:
%pip install -q wurlitzer ninja
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
%load_ext wurlitzer

import torch
from torch.utils.cpp_extension import load_inline

The wurlitzer extension is already loaded. To reload it, use:
  %reload_ext wurlitzer


In [15]:
# Utility fucntions for CUDA

def load_cuda(cuda_src, cpp_src, funcs, opt=True, verbose=False, name=None):
    "Simple wrapper for torch.utils.cpp_extension.load_inline"
    if name is None: name = funcs[0]
    flags = "-O3 -Xptxas -O3 -Xcompiler -O3" if opt else "-O0 -Xptxas -O0 -Xcompiler -O0"
    return load_inline(cuda_sources=[cuda_src], cpp_sources=[cpp_src], functions=funcs,
                       extra_cuda_cflags=[flags], verbose=verbose, name=name)

# Utility code strings for CUDA

cuda_begin = r'''
#include <torch/extension.h>
#include <stdio.h>
#include <c10/cuda/CUDAException.h>

#define CHECK_CUDA(x) TORCH_CHECK(x.device().is_cuda(), #x " must be a CUDA tensor")
#define CHECK_CONTIGUOUS(x) TORCH_CHECK(x.is_contiguous(), #x " must be contiguous")
#define CHECK_INPUT(x) CHECK_CUDA(x); CHECK_CONTIGUOUS(x)
#define CUDA_ERR(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess)
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}
__host__ __device__ inline unsigned int cdiv(unsigned int a, unsigned int b) { return (a+b-1)/b;}
'''

cuda_end = r'''
torch::Tensor mat_mul(torch::Tensor A, torch::Tensor B) {
    CHECK_INPUT(A); CHECK_INPUT(B);
    int rows=A.size(0), columns=B.size(1), inners=A.size(1);
    TORCH_CHECK(inners==B.size(0), "Size mismatch!");

    auto C = torch::zeros({rows, columns}, A.options());
    dim3 tpb(16, 16);
    dim3 blocks(cdiv(rows, tpb.x), cdiv(columns, tpb.y));

    mat_mul_naive<<<blocks, tpb>>>(
      static_cast<float*>(A.data_ptr()), static_cast<float*>(B.data_ptr()), static_cast<float*>(C.data_ptr()),
      rows, columns, inners);
    C10_CUDA_KERNEL_LAUNCH_CHECK();
    return C;
}
'''

cpp4python = "torch::Tensor mat_mul(torch::Tensor A, torch::Tensor B);"

#TODO: add f string support to these to automate code gen.

In [16]:
# Niave kernel

mat_mul_naive = r'''
__global__ void mat_mul_naive(const float *A, const float *B, float *C, int rows, int columns, int inners) {

  // C = A @ B : (rows X inners) @ (inners X columns) -> (rows X columns)

  const uint x = blockIdx.x * blockDim.x + threadIdx.x;
  const uint y = blockIdx.y * blockDim.y + threadIdx.y;

  // if statement is necessary for when rows or columns aren't multiples of the CUDA block size.

  if (x < columns && y < rows) {
    float tmp = 0.0;
    for (int i = 0; i < inners; ++i) {tmp += A[x * inners + i] * B[i * columns + y];}
    C[x * columns + y] = tmp;
  }
}
'''

In [17]:
cuda_code = cuda_begin + mat_mul_naive + cuda_end

In [18]:
prg = load_cuda(cuda_src=cuda_code, cpp_src=cpp4python, funcs=['mat_mul'])

In [77]:
rows = 1024
columns = 1024
inners = 1024
flop = (2 * inners) * rows * columns
A = torch.rand(rows, inners, dtype=torch.float32, device='cuda')
B = torch.rand(inners, columns, dtype=torch.float32, device='cuda')

In [78]:
A.is_contiguous(), B.is_contiguous()

(True, True)

In [79]:
torch.isclose(A@B, prg.mat_mul(A, B)).all()

tensor(True, device='cuda:0')

How to time CPU/GPU events : https://developer.nvidia.com/blog/how-implement-performance-metrics-cuda-cc/

In [96]:

se = torch.cuda.Event(enable_timing=True)
ee = torch.cuda.Event(enable_timing=True)

# Record the start time
se.record()

# Perform the matrix multiplication

# C = torch.matmul(A, B)
C = prg.mat_mul(A, B)

# Record the end time
ee.record()

# Block CPU execution until the event is recorded.
torch.cuda.synchronize()

t = se.elapsed_time(ee)

print(f'GFLOPs: {flop / t*1e3 / 1e9: .6f} | Time {t:.4f} ms')

GFLOPs:  43.188403 | Time 49.7236 ms


In [ ]:
# cuBLAS: 1600 GFLOPS (1.3ms)
# openBLAS: 110 GFLOPS (20ms), 2 CPU cores

# niave CUDA: 43 GFLOPS (49ms)

In [81]:
import numpy as np
Anp = np.random.rand(rows, inners).astype(np.float32)
Bnp = np.random.rand(inners, columns).astype(np.float32)

In [91]:
Acpu = A.cpu()
Bcpu = B.cpu()
A.device, Acpu.device

(device(type='cuda', index=0), device(type='cpu'))

In [92]:
%timeit -n 10 _ = Acpu @ Bcpu

19.1 ms ± 1.63 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [93]:
%timeit -n 10 _ = Anp @ Bnp

20.7 ms ± 1.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [51]:
np.show_config()

openblas64__info:
    libraries = ['openblas64_', 'openblas64_']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None), ('BLAS_SYMBOL_SUFFIX', '64_'), ('HAVE_BLAS_ILP64', None)]
    runtime_library_dirs = ['/usr/local/lib']
blas_ilp64_opt_info:
    libraries = ['openblas64_', 'openblas64_']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None), ('BLAS_SYMBOL_SUFFIX', '64_'), ('HAVE_BLAS_ILP64', None)]
    runtime_library_dirs = ['/usr/local/lib']
openblas64__lapack_info:
    libraries = ['openblas64_', 'openblas64_']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None), ('BLAS_SYMBOL_SUFFIX', '64_'), ('HAVE_BLAS_ILP64', None), ('HAVE_LAPACKE', None)]
    runtime_library_dirs = ['/usr/local/lib']
lapack_ilp64_opt_info:
    libraries = ['openblas64_', 'openblas64_']
    library_dirs = ['/usr/local/lib']
    language = c
    define_macros = [('HAVE_CBLAS', None

In [52]:
os.cpu_count()

2